In [1]:
from lightning_simclr_cifar10 import SimCLR, eval

paths = [
    "version_1",
    # "version_2",
    "version_3",
    "version_4",
    "version_5"
]
name = "/checkpoints/epoch=999-step=59000.ckpt"
paths = ["logs/lightning_logs/"+path for path in paths]


for path in paths:
    model = SimCLR.load_from_checkpoint(path+name)
    eval(model, path)

Computing features for evaluation
kNN accuracy (Euclidean): 0.9146
kNN accuracy (cosine): 0.9175
kNN accuracy (cosine), Z: 0.9088
kNN accuracy (cosine), Z after normalization: 0.9110
Linear accuracy (saga/no-penalty): 0.9291


/gpfs01/berens/user/jdoehl/tsimcne/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Computing features for evaluation
kNN accuracy (Euclidean): 0.904
kNN accuracy (cosine): 0.9026
kNN accuracy (cosine), Z: 0.8936
kNN accuracy (cosine), Z after normalization: 0.8989
Linear accuracy (saga/no-penalty): 0.9188


/gpfs01/berens/user/jdoehl/tsimcne/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Computing features for evaluation
kNN accuracy (Euclidean): 0.8308
kNN accuracy (cosine): 0.8308
kNN accuracy (cosine), Z: 0.8200
kNN accuracy (cosine), Z after normalization: 0.8268
Linear accuracy (saga/no-penalty): 0.8807


/gpfs01/berens/user/jdoehl/tsimcne/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Computing features for evaluation
kNN accuracy (Euclidean): 0.9138
kNN accuracy (cosine): 0.9162
kNN accuracy (cosine), Z: 0.9089
kNN accuracy (cosine), Z after normalization: 0.9089
Linear accuracy (saga/no-penalty): 0.9269


/gpfs01/berens/user/jdoehl/tsimcne/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
